In [1]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [129]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from datetime import datetime,time,date
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os 
import h5py
import talib

In [3]:
# Import My own library for factor testing
from SingleFactorTest import factorFilterFunctions as ff
#from config import *
from SingleFactorTest.calcOwnFactors import CalOwnFactor

In [4]:
%load_ext line_profiler

In [5]:
path = ff.data_path # path

In [6]:
# --------------------------------------- Global Functions to def---------------------------------- #

In [123]:
class neutralize(CalOwnFactor):
    classname = 'neutralize'
    
    def __init__(self, path,filenameFCAP,factorData):
        super(neutralize,self).__init__(path) # arguments in super method should be exactly same as the args in parent class
        self.logFCAP = np.log10(ff.readh5data(path,filenameFCAP))
        self.factorData = factorData
        
    def simpleNormalize(self,factorData):
        factorDatatemp = factorData.copy()
        dataWinsorizedTrans = factorDatatemp.T
        MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
        return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/\
                               dataWinsorizedTrans.std(axis=0, skipna=True)).T
    def neutralizeFactor(self, datelist):
        normalizedLFCAPDF = self.simpleNormalize(self.logFCAP)
        normalizedFactor= self.simpleNormalize(self.factorData)
        factorNeutralized = pd.DataFrame(index=normalizedFactor.index, columns=normalizedFactor.columns, data=None, dtype = float)
        for date in datelist:
            LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
            factorIndice = normalizedFactor.loc[date].dropna()
            intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
            #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
            #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
            try:
                result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
                factorNeutralized.loc[date][intersectionStocks] = result.resid
            except:
                factorNeutralized.loc[date] = np.NaN
        self.factorNeutralized = factorNeutralized.round(4)

In [124]:
neutralized = neutralize(path,filenameFCAP,illiq.ILLIQ)
neutralized.neutralizeFactor(neutralized.factorData.index)


c:\python27\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: invalid value encountered in absolute


In [127]:
neutralized.factorNeutralized.tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300705.SZ,300707.SZ,603103.SH,002903.SZ,002906.SZ,603110.SH,002905.SZ,300708.SZ,603499.SH,603829.SH
date,,,,,,,,,,,,,,,,,,,,,
2017-10-10,-0.0555,-0.0296,-0.0300,-0.0352,-0.0423,-0.0502,-0.0630,NaN,0.1241,0.1491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-11,-0.0664,-0.0417,-0.0390,-0.0430,-0.0524,-0.0636,-0.0761,NaN,0.1840,0.1387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-12,-0.0938,-0.0557,-0.0531,-0.0553,-0.0723,-0.0844,-0.1007,NaN,0.1767,0.1948,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-13,-0.5567,-0.3979,-0.3660,-0.3302,-0.4622,-0.5464,-0.5980,NaN,1.1585,0.9693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


649.9999999999999

In [7]:
# top functions to def
def simpleNormalize(narrowedData):
    dataWinsorized = narrowedData.copy()
    dataWinsorizedTrans = dataWinsorized.T
    MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
    return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/dataWinsorizedTrans.std(axis=0, skipna=True)).T

In [8]:
# top function
def neutralizeFactor(normalizedFactorDF, normalizedLFCAPDF, datelist):
    factorNeutralized = pd.DataFrame(index=normalizedFactorDF.index, columns=normalizedFactorDF.columns, data=None, dtype = float)
    for date in datelist:
        LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
        factorIndice = normalizedFactorDF.loc[date].dropna()
        intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
        #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
        #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
        try:
            result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
            factorNeutralized.loc[date][intersectionStocks] = result.resid
        except:
            factorNeutralized.loc[date] = np.NaN
    return factorNeutralized

In [9]:
# --------------------------------------- Function Section End ---------------------------------- #

In [95]:
filenameFCAP =  'LZ_CN_STKA_VAL_A_FCAP.h5'
# Data prepared for Neuralization
FCAP1 = np.log10(ff.readh5data(path,filenameFCAP))
NormalizedFCAP = simpleNormalize(FCAP1)

c:\python27\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in absolute


In [ ]:
# --------------------------------------- AdjustedPrice ---------------------------------- #

In [23]:
filenameAdjustFactor = 'LZ_CN_STKA_CMFTR_CUM_FACTOR.h5'
filenameClose = 'LZ_CN_STKA_QUOTE_TCLOSE.h5'

In [52]:
# first thing is to calculate forward adjusted pice
class AdjustedPrice(CalOwnFactor):
    classname = 'AdjustedPrice'
    def __init__(self,path):
        super(AdjustedPrice,self).__init__(path) # arguments in super method should be exactly same as the args in parent class
        #self.setting = setting
    def calAdjustedPrice(self,adjFacBackward,originalPrice):
        AdjFacforward = adjFacBackward / adjFacBackward.max()
        self.adjustedPrice = (AdjFacforward * originalPrice).round(3)     

In [53]:
adjc = AdjustedPrice(path)
adjc.addData('adjf',filenameAdjustFactor)
adjc.addData('close',filenameClose)
adjc.calAdjustedPrice(adjc.datadict['adjf'],adjc.datadict['close'])
adjc.saveData(adjc.adjustedPrice,'OwnfactorAdjustedClose')

In [ ]:
# --------------------------------------- end  AdjustedPrice section ---------------------------------- #

In [12]:
# --------------------------------------- Mass Index ---------------------------------- #

In [71]:
filenameOpen = 'LZ_CN_STKA_QUOTE_TOPEN.h5' # 开盘价
filenameClose = 'LZ_CN_STKA_QUOTE_TCLOSE.h5'  # 收盘价
filenameHigh = 'LZ_CN_STKA_QUOTE_THIGH.h5'   # 最高价
filenameLow = 'LZ_CN_STKA_QUOTE_TLOW.h5'  # 最低价
# 

filenameVolume = 'LZ_CN_STKA_QUOTE_TVOLUME.h5'

In [14]:
class MassIndex(CalOwnFactor):
    classname = 'MassIndex'
    def __init__(self,path):
        super(MassIndex,self).__init__(path) # arguments in super method should be exactly same as the args in parent class
        #self.setting = setting
    def calEma(self,data,window = 9):
        Ema = data.copy()
        NanCount =0
        for name in data.columns:
            try:
                Ema[name] = talib.EMA(data[name].values,timeperiod = window)
            except Exception as e:
                #print str(e) 
                #print 'too much NaN value for stock %s'%name
                assert str(e)== 'inputs are all NaN'
                Ema [name] = np.NaN
                NanCount+=1
        return Ema
    def calMassIndex(self,hlRange,rolling_window=25):
        self.singleEMA = self.calEma(hlRange)
        self.doubleEMA = self.calEma(self.singleEMA)
        self.MassIndex = (self.singleEMA / self.doubleEMA).rolling(window=rolling_window, min_periods=25).sum().round(3)

In [15]:
# cal massindex
massindex = MassIndex(path)
massindex.addData('high',filenameHigh)
massindex.addData('low',filenameLow)

massindex.calMassIndex(massindex.datadict['high'] - massindex.datadict['low'])

massindex.saveData(massindex.MassIndex,'OwnfactorMassIndex') 

c:\python27\lib\site-packages\pandas\core\series.py:1295: RuntimeWarning: invalid value encountered in rint
  result = _values_from_object(self).round(decimals)


In [16]:
# --------------------------------------- end Mass Index  section ---------------------------------- #

In [ ]:
# --------------------------------------- ILLQ Factor(5-days average) ---------------------------------- #

In [81]:
# the illiquid factor
class ILLIQ(CalOwnFactor):
    classname = 'ILLIQ'
    def __init__(self,path):
        super(ILLIQ,self).__init__(path) # arguments in super method should be exactly same as the args in parent class
        #self.setting = setting
    def calILLIQ(self,openPrice,closePrice,volume):
        if openPrice.shape != closePrice.shape:
            print openPrice.shape, closePrice.shape
            print 'data shape is not equal!'
        else:
            newdf = np.abs((closePrice - openPrice)/openPrice)/volume
            self.ILLIQ= (newdf.rolling(min_periods=5,window=5,center=False).mean()* 10000000).round(4)

In [82]:
# cal ILLIQ
illiq = ILLIQ(path)
illiq .addData('open',filenameOpen)
illiq .addData('close',filenameClose)
illiq .addData('volume', filenameVolume)
illiq .calILLIQ(illiq.datadict['open'],illiq.datadict['close'],illiq.datadict['volume'])
illiq .saveData(illiq.ILLIQ,'OwnFactorILLIQ') 

c:\python27\lib\site-packages\ipykernel\__main__.py:12: RuntimeWarning: invalid value encountered in absolute


In [99]:
# TO nuetralize ILLIQ
NormalizedILLQ = simpleNormalize(illiq.ILLIQ)
neutralizedILLQ = neutralizeFactor(NormalizedILLQ, NormalizedFCAP, NormalizedILLQ.index)
#neutralizedILLQ.index.name = 'Own_Factor_ADJ_ILLQ_1D'
ff.saveh5data(neutralizedILLQ,path,'OwnFactorADJILLIQ')

c:\python27\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in absolute


In [84]:
# --------------------------------------- end ILLQ Factor section ---------------------------------- #